In [2]:
import socket
import hashlib
import json
import time
import matplotlib.pyplot as plt
import os
import csv

In [2]:
HOST = '127.0.0.1'    
PORT = 13854
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))

In [3]:
# # Authenticate
# client_phrase =b"I am a woman with great aspirations"
# client_key = hashlib.sha1(client_phrase).hexdigest()

# app_name = "Suchi EEG"

# auth_dict = {"appName":app_name,"appKey":client_key}

# auth_msg = json.dumps(auth_dict).encode('ascii')
# s.send(auth_msg)

In [4]:
#set a server config and validate
server_config = json.dumps({"enableRawOutput": True, "format": "Json"}).encode('ascii')
s.send(server_config)

43

In [5]:
def get_data(secs=120):
    event=input('Enter \n1 for Baseline \n2 for Game\n or 0 for other \n')
    
    
    if int(event)==2:
            event_name='Game_'
    elif int(event)==1:
            event_name='Baseline_'
    else:
        event_name='Other_'
    
    if not(os.path.exists('data')):
        print('Creating folder: data..')
        os.makedirs('data')
    timet=time.strftime("%d-%m_%H-%M-%S")
    logdir =event_name+timet +'.json'
    logdir = os.path.join('data', logdir)
    feeling=input('How are you feeling? ')
    task=input('Is there any specific task name? ')
#     secs=input('Input recording time: ')
 
    
    with open(logdir, 'w', encoding="ascii", newline='\r\n') as outfile:
        ref = time.time()
        while (time.time() - ref) < secs:
#         else:
#             print (ref)
#             try:
            response = s.recv(2048).decode('ascii')
#             print((time.time() - ref))
            outfile.write(response)
#             print(">>>", response)
        
#             except:
#                 pass
            
            
        metadata={"metadata":{"Feeling":feeling,"Time":timet,"Task":task}}
        outfile.write(json.dumps(metadata))

    return logdir



In [6]:
# print (time.localtime( time.time() ))

In [7]:


def plot_raw_from_file(fpath):
    print(fpath)
    eegs = []
    for line in open(fpath,'r'):
        line=line.strip()
        if line!="":
            raw_eeg = json.loads(line)
            if 'rawEeg' in raw_eeg:
                eegs.append(raw_eeg['rawEeg'])
#             if 'metadata' in raw_eeg:
#                 print(raw_eeg)
                
    fig = plt.figure(figsize=(15,4))
    plt.plot(eegs,'r')
    plt.title(fpath)
    plt.show()
#     return eegs

In [8]:
def get_data_f(secs=30):
    event=input('Enter \n1 for Baseline \n2 for Game\n or 0 for other \n')
    
    
    if int(event)==2:
            event_name='Game_'
    elif int(event)==1:
            event_name='Baseline_'
    else:
        event_name='Other_'
    
    if not(os.path.exists('data')):
        print('Creating folder: data..')
        os.makedirs('data')
    timet=time.strftime("%d-%m_%H-%M-%S")
    logdir =event_name+timet +'.json'
    logdir = os.path.join('data', logdir)
    feeling=input('How are you feeling? ')
    task=input('Is there any specific task name? ')
#     secs=input('Input recording time: ')
    with open(logdir, 'w', encoding="ascii", newline='\r\n') as outfile:
        ref = time.time()
        while (time.time() - ref) < secs:
            response = s.recv(2048).decode('ascii')
    #         print(json.dumps(response))
            outfile.write(response)
        
        metadata={"metadata":{"Feeling":feeling,"Time":timet,"Task":task}}
        outfile.write(json.dumps(metadata))

     

In [ ]:

file=get_data_f(secs=20)
file



Enter 
1 for Baseline 
2 for Game
 or 0 for other 
1
How are you feeling? 11
Is there any specific task name? 11


In [ ]:
 plot_raw_from_file(file)

# plot_raw_from_file(file)

In [3]:
import pandas as pd

path='data'
files=!ls data
# files
df=pd.DataFrame(columns=['eeg','activity','date','time','metadata'])
for file in files:
    print(file)
    if file.endswith('.json'):
        eegs = []
        filename=path+'/'+file
        for line in open(filename,'r'):
            line=line.strip()
            if line!="":
                try:
                    raw_eeg = json.loads(line)
                    if 'rawEeg' in raw_eeg:
                        eegs.append(raw_eeg['rawEeg'])
                    if 'metadata' in raw_eeg:
                        metadata=raw_eeg
                except:
                    print('\n>>>ERROR: cant read a line in',file)
                    pass
        filesplit=file.split('_')
        
        activity=filesplit[0]
        date=filesplit[1]
#         time=filesplit[2][:-5]
    #     print(activity,date,time,metadata)
        df2=pd.DataFrame([[eegs,activity,date,time,metadata]],columns=['eeg','activity','date','time','metadata'])
        df=pd.concat([df2,df])      
    


Baseline_04-12_12-42-54.json

>>>ERROR: cant read a line in Baseline_04-12_12-42-54.json


NameError: name 'metadata' is not defined

In [10]:
!ls ../


MindReading             data_after_29_oct.csv   fft_500_28_nov.pdf
OSX Developer Tools 3.3 data_till_29_oct.csv    test
data                    eegs_500_28_nov.pdf


In [15]:
df.shape
path='../'
dataset = pd.read_csv(path+'data_after_29_oct.csv')
dataset.eeg = dataset.eeg.map(json.loads)
# dataset['date']=pd.to_datetime(dataset['date'])

In [14]:
df.to_csv('data_after_29_oct.csv',index=False)

In [16]:
dataset.shape

(21, 5)

In [17]:
dataset

,eeg,activity,date,time,metadata
0,"[-28, -20, -36, 58, -3, 37, 65, 163, 203, -49,...",Other,30-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'normala', 'Time': '3..."
1,[],Other,22-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': '', 'Time': '31-10_09..."
2,"[151, 108, 74, 184, 167, 153, 280, 144, -21, 1...",Game,31-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': '', 'Time': '31-10_09..."
3,"[90, 55, 37, 109, 33, 39, 49, 41, 67, -5, 7, 4...",Game,30-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'normal', 'Time': '30..."
4,"[99, -108, 56, 173, 170, 210, 200, 160, 100, 1...",Game,12-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'ee', 'Time': '12-11_..."
5,"[70, 184, -18, -27, 36, -5, 17, 67, 54, 59, 10...",Game,12-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'n_back', 'Time': '12..."
6,"[329, 698, 648, 83, -201, -306, 61, 435, 225, ...",Game,10-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'exam', 'Time': '10-1..."
7,"[-533, -353, 2047, 1389, 970, 682, 484, 348, -...",Game,07-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'tired and anxious', ..."
8,"[59, -11, 37, 135, 235, 224, 315, 263, 297, 27...",Game,06-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'okk', 'Time': '06-11..."
9,"[342, -3, -107, -52, -3, 38, 6, 37, 55, 96, 52...",Game,02-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'tired', 'Time': '02-..."


In [22]:
lengths_of_eeg=dataset['eeg'].apply(lambda x:len(x))
min(lengths_of_eeg)

rm=[i for i, j in enumerate(lengths_of_eeg) if j == 0]
rm# # drop any eeg of len 0
dataset=dataset.drop(dataset.index[[rm]])
# lengths_of_eeg=dataset['eeg'].apply(lambda x:len(x))
# min(lengths_of_eeg)
# # drop any eeg of len 0
# dataset=dataset.drop(dataset.index[[rm]])
# lengths_of_eeg=dataset['eeg'].apply(lambda x:len(x))
# min(lengths_of_eeg)

In [23]:
dataset

,eeg,activity,date,time,metadata
0,"[-28, -20, -36, 58, -3, 37, 65, 163, 203, -49,...",Other,30-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'normala', 'Time': '3..."
2,"[151, 108, 74, 184, 167, 153, 280, 144, -21, 1...",Game,31-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': '', 'Time': '31-10_09..."
3,"[90, 55, 37, 109, 33, 39, 49, 41, 67, -5, 7, 4...",Game,30-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'normal', 'Time': '30..."
4,"[99, -108, 56, 173, 170, 210, 200, 160, 100, 1...",Game,12-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'ee', 'Time': '12-11_..."
5,"[70, 184, -18, -27, 36, -5, 17, 67, 54, 59, 10...",Game,12-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'n_back', 'Time': '12..."
6,"[329, 698, 648, 83, -201, -306, 61, 435, 225, ...",Game,10-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'exam', 'Time': '10-1..."
7,"[-533, -353, 2047, 1389, 970, 682, 484, 348, -...",Game,07-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'tired and anxious', ..."
8,"[59, -11, 37, 135, 235, 224, 315, 263, 297, 27...",Game,06-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'okk', 'Time': '06-11..."
9,"[342, -3, -107, -52, -3, 38, 6, 37, 55, 96, 52...",Game,02-11,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'tired', 'Time': '02-..."
10,"[188, 182, 168, 117, 88, 69, 57, 54, 51, 50, 4...",Baseline,31-10,<module 'time' (built-in)>,"{'metadata': {'Feeling': 'blank', 'Time': '31-..."
